### Import

In [18]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, MultiLabelBinarizer
from sklearn.ensemble import RandomForestClassifier

### Data Load

In [19]:
train = pd.read_csv('selected_features_cleaned.csv').drop(columns=['ID'])
test = pd.read_csv('test.csv').drop(columns=['ID'])

missing_cols = set(test.columns) - set(train.columns)
test = test.drop(columns=missing_cols)

In [20]:
X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

### Data Pre-Processing

In [21]:
# ordinal 인코딩 필요 컬럼
ordinal_columns = [
    "시술 시기 코드", "시술 당시 나이", "총 시술 횟수", "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수", "DI 시술 횟수", "총 임신 횟수", "IVF 임신 횟수", "DI 임신 횟수",
    "총 출산 횟수", "IVF 출산 횟수", "DI 출산 횟수", "난자 기증자 나이", "정자 기증자 나이"
]

# one-hot 인코딩 필요 컬럼
onehot_columns = ["시술 유형"]

# multi label 인코딩 필요 컬럼
multi_label_columns = ["특정 시술 유형", "배아 생성 주요 이유", "난자 출처", "정자 출처"]

    
# 범주형 데이터 문자열 변환
categorical_columns = ordinal_columns + onehot_columns + multi_label_columns

for col in categorical_columns:
    X[col] = X[col].astype(str)
    test[col] = test[col].astype(str)  # 테스트 데이터도 동일하게 변환


In [22]:
X_original = X.copy()
test_original = test.copy()

# 1️. Ordinal Encoding
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X_encoded = X.copy()
test_encoded = test.copy()

X_encoded[ordinal_columns] = ordinal_encoder.fit_transform(X[ordinal_columns])
test_encoded[ordinal_columns] = ordinal_encoder.transform(test[ordinal_columns])


# 2️. One-Hot Encoding
onehot_encoder = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')

onehot_df = pd.DataFrame(onehot_encoder.fit_transform(X_encoded[onehot_columns]),
                         columns=onehot_encoder.get_feature_names_out(onehot_columns))
test_onehot_df = pd.DataFrame(onehot_encoder.transform(test_encoded[onehot_columns]),
                              columns=onehot_encoder.get_feature_names_out(onehot_columns))

# 기존 컬럼 제거 및 변환된 데이터 추가
X_encoded.drop(columns=onehot_columns, inplace=True)
X_encoded = pd.concat([X_encoded, onehot_df], axis=1)
test_encoded.drop(columns=onehot_columns, inplace=True)
test_encoded = pd.concat([test_encoded, test_onehot_df], axis=1)


# 3️. Multi-Label One-Hot Encoding 적용
for col in multi_label_columns:
    X_encoded[col] = X_encoded[col].apply(lambda x: eval(x) if isinstance(x, str) and x.startswith("[") else [x])
    test_encoded[col] = test_encoded[col].apply(lambda x: eval(x) if isinstance(x, str) and x.startswith("[") else [x])
    
    mlb = MultiLabelBinarizer()
    X_mlb = mlb.fit_transform(X_encoded[col])
    test_mlb = mlb.transform(test_encoded[col])
    
    multi_df = pd.DataFrame(X_mlb, columns=[f"{col}_{c}" for c in mlb.classes_])
    test_multi_df = pd.DataFrame(test_mlb, columns=[f"{col}_{c}" for c in mlb.classes_])
    
    X_encoded.drop(columns=[col], inplace=True)
    X_encoded = pd.concat([X_encoded, multi_df], axis=1)
    test_encoded.drop(columns=[col], inplace=True)
    test_encoded = pd.concat([test_encoded, test_multi_df], axis=1)

# 최종 인코딩된 데이터셋 출력
print(X_encoded.head())
print(test_encoded.head())

C:\Users\서혜린\AppData\Roaming\Python\Python313\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\서혜린\AppData\Roaming\Python\Python313\site-packages\sklearn\preprocessing\_label.py:909: UserWarning: unknown class(es) ['Generic DI', 'ICI', 'IUI', 'IUI:ICI', 'IVF:Unknown:Unknown:Unknown', 'IVI', 'Unknown'] will be ignored
  warnings.warn(
C:\Users\서혜린\AppData\Roaming\Python\Python313\site-packages\sklearn\preprocessing\_label.py:909: UserWarning: unknown class(es) ['nan', '기증용', '기증용, 난자 저장용', '기증용, 배아 저장용', '난자 저장용', '난자 저장용, 배아 저장용', '배아 저장용'] will be ignored
  warnings.warn(
C:\Users\서혜린\AppData\Roaming\Python\Python313\site-packages\sklearn\preprocessing\_label.py:909: UserWarning: unknown class(es) ['기증 제공', '알 수 없음'] will be ignored
  warnings.warn(


   시술 시기 코드  시술 당시 나이  배란 자극 여부  단일 배아 이식 여부  착상 전 유전 진단 사용 여부  남성 주 불임 원인  \
0       3.0       0.0         1          0.0               0.0           0   
1       4.0       3.0         1          0.0               0.0           0   
2       5.0       0.0         1          0.0               0.0           0   
3       1.0       4.0         1          0.0               0.0           0   
4       6.0       0.0         1          0.0               0.0           0   

   남성 부 불임 원인  여성 주 불임 원인  여성 부 불임 원인  부부 주 불임 원인  ...  특정 시술 유형_IVF:IVF  \
0           0           0           0           0  ...                 0   
1           0           0           0           0  ...                 0   
2           0           0           0           0  ...                 0   
3           0           0           0           0  ...                 0   
4           0           0           0           0  ...                 0   

   특정 시술 유형_IVF:Unknown  배아 생성 주요 이유_기증용, 배아 저장용, 현재 시술용  \
0             

C:\Users\서혜린\AppData\Roaming\Python\Python313\site-packages\sklearn\preprocessing\_label.py:909: UserWarning: unknown class(es) ['미할당', '배우자 및 기증 제공'] will be ignored
  warnings.warn(


### Train

In [23]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_encoded, y)

RandomForestClassifier(random_state=42)

In [24]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

# 80% 훈련, 20% 검증 데이터로 분리
X_train, X_val, y_train, y_val = train_test_split(X_encoded, y, test_size=0.2, random_state=42, stratify=y)

# 모델 학습
#model = RandomForestClassifier(n_estimators=100, random_state=42)
#model = RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_split=10, min_samples_leaf=5, random_state=42)
model = RandomForestClassifier(n_estimators=300, max_depth=10, min_samples_split=10, min_samples_leaf=5, random_state=42)
model.fit(X_train, y_train)

# 검증 데이터에 대한 성능 평가
y_val_pred = model.predict(X_val)
y_val_proba = model.predict_proba(X_val)[:, 1]

accuracy = accuracy_score(y_val, y_val_pred)
roc_auc = roc_auc_score(y_val, y_val_proba)

print(f"Validation Accuracy: {accuracy:.4f}")
print(f"Validation ROC-AUC Score: {roc_auc:.4f}")

# 5-Fold Cross Validation
cv_scores = cross_val_score(model, X_encoded, y, cv=5, scoring="roc_auc")

print(f"Cross-Validation ROC-AUC Scores: {cv_scores}")
print(f"Mean ROC-AUC Score: {cv_scores.mean():.4f}")

Validation Accuracy: 0.7416
Validation ROC-AUC Score: 0.6622
Cross-Validation ROC-AUC Scores: [0.65967603 0.66626319 0.65418366 0.67867626 0.66767195]
Mean ROC-AUC Score: 0.6653


RandomForest (Hyperparameter tune)

### Predict

In [27]:
pred_proba = model.predict_proba(test_encoded)[:, 1]

### Submission

In [28]:
sample_submission = pd.read_csv('sample_submission.csv')
sample_submission['probability'] = pred_proba

In [29]:
sample_submission.to_csv('./encoding_test_submit2.csv', index=False)